In [1]:
%%writefile train_model.py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import json
import os

# Define the neural network architecture
class ScoringModel(nn.Module):
    def __init__(self, num_inputs=29):
        super(ScoringModel, self).__init__()
        self.layer1 = nn.Linear(num_inputs, 512)
        self.layer2 = nn.Linear(512, 256)
        self.layer3 = nn.Linear(256, 128)
        self.layer4 = nn.Linear(128, 64)
        self.layer5 = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 1)
        self.leaky_relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.leaky_relu(self.layer1(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer2(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer3(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer4(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer5(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x

# Function to load experiences from JSON file
def load_experiences(path):
    with open(path, 'r') as f:
        experiences_json = json.load(f)
    
    experiences = []
    for exp in experiences_json:
        experiences.append((torch.tensor(exp['state']), exp['action'], exp['reward'], torch.tensor(exp['next_state']), exp['done']))
    return experiences

# Function to prepare data from experiences
def prepare_data(experiences):
    states = [exp[0] for exp in experiences]
    actions = [exp[1] for exp in experiences]
    rewards = [exp[2] for exp in experiences]
    next_states = [exp[3] for exp in experiences]
    dones = [exp[4] for exp in experiences]
    
    states = torch.stack(states)
    actions = torch.tensor(actions)
    rewards = torch.tensor(rewards)
    next_states = torch.stack(next_states)
    dones = torch.tensor(dones)
    
    return states, actions, rewards, next_states, dones

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function for fine-tuning the model
def train_model(train_loader, model, criterion, optimizer, num_epochs=500):
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        if (epoch + 1) % 10 == 0:  # Print every 10 epochs
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

# Main function to load experiences and train the model
def main():
    try:
        experiences_path = "/Users/marcel/Desktop/project/model/experiences.json"
        model_path = "/Users/marcel/Desktop/project/model/soccer_ai_model.pt"

        print("Loading experiences...")
        # Load experiences
        experiences = load_experiences(experiences_path)
        print("Experiences loaded successfully.")

        print("Preparing data...")
        # Prepare data
        states, actions, rewards, next_states, dones = prepare_data(experiences)
        print("Data prepared successfully.")

        # Convert to PyTorch DataLoader
        dataset = TensorDataset(states, rewards)  # Assuming rewards are used as labels for simplicity
        train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

        # Initialize the model
        model = ScoringModel(num_inputs=29)

        # Load the model state if it exists
        if os.path.isfile(model_path):
            print("Loading existing model state...")
            model.load_state_dict(torch.load(model_path))
            print("Model state loaded successfully.")

        # Train the model
        print("Training the model...")
        train_model(train_loader, model, criterion, optimizer)
        print("Model trained successfully.")

        # Save the updated model state_dict
        print("Saving the model state...")
        torch.save(model.state_dict(), model_path)
        print("Model state saved successfully.")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


Overwriting train_model.py
